# Running inference

In [1]:
import librosa as lr
import octafilt3r.filter as o3f
import os
from tensorflow.keras.models import load_model
import numpy as np
import pipeline.config as conf
import pipeline.scale as scale

In [2]:
param_dict = conf.open_params()

base_path = "Datasets/UrbanSound8k_augmented/street_music/"
stop_at = 30
i = 0
true = 0
false = 0

N4CED = load_model(param_dict["CNN_name"])

for file in os.listdir(base_path):

    if(i == stop_at):
        break

    """
    LOAD DATA
    """
    wav, fs = lr.load(base_path + file, sr=param_dict['fs'], duration=param_dict["max_dur"])

    if len(wav) < fs:
        continue


    """
    EXTRACT FEATURES
    """
    feats, fcs = o3f.rolling_oct_bank(
        wav,
        fs,
        param_dict['oct_bw_ratio'],
        param_dict['order'],
        param_dict['fmax'],
        param_dict['fmin'],
        param_dict['frame_size'],
        param_dict['dec_stages'],
        param_dict['dec_ord']
    )


    """
    RESHAPE FEATURES
    """
    df = scale.feats2frames(feats, param_dict['frame_size'], fs)


    """
    SCALE FEATURES
    """
    X, scaler = scale.data_scaler(df)


    """
    RUN MODEL
    """
    p = N4CED.predict(X)


    """
    PROCESS RESULTS
    """
    predicted = param_dict["class_map"][np.argmax(np.mean(p, axis=0))]
    print(f'Predicted class: "{predicted}" with {int(100 * (p[0][np.argmax(p, 1)][0]))} score.')
    if predicted == 'street_music':
        true += 1
    else:
        false += 1
    i += 1

print('----- done. -----')
print(f'{true} true predictions\n{false} false predictions')
print(f'test accuracy: {int((true/i)*100)}%\nerror ratio: {int((false/i)*100)}%')

[[[-112.07846872 -101.86949262  -96.17280617 ...  -58.9812725
    -60.13743646  -61.95149668]
  [ -92.09075843  -85.96212331  -81.96860828 ...  -66.42813554
    -60.36534606  -61.96858903]
  [ -80.7387277   -76.04074525  -72.43306545 ...  -62.86805655
    -61.19527704  -61.2146005 ]
  ...
  [ -71.07396329  -51.66228852  -53.79138514 ...  -60.73648664
    -60.00894919  -55.77278697]
  [ -69.98841126  -49.97433937  -49.7568959  ...  -60.24063736
    -58.57247969  -58.62413913]
  [ -64.91913955  -49.75660271  -46.04863918 ...  -59.64576168
    -58.08972493  -56.94516303]]

 [[ -61.47995462  -48.01947404  -46.1688925  ...  -60.07963354
    -59.88114422  -59.61316483]
  [ -59.22492021  -49.53146089  -47.15983842 ...  -55.92916647
    -61.0897809   -60.63917594]
  [ -57.82733408  -52.07931623  -48.55707018 ...  -63.04381432
    -61.62077401  -60.83209102]
  ...
  [ -60.45849406  -55.97000498  -54.70351029 ...  -61.46069836
    -61.23335613  -64.92786635]
  [ -60.00864956  -58.55187395  -57.0

In [3]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(N4CED)
tflite_model = converter.convert()
open(param_dict["CNN_name"] + "/" + param_dict["CNN_name"] + "_lite.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\JDA1E~1.TSC\AppData\Local\Temp\tmp55z4slj7\assets


745868